In [1]:
import torch
import torch.nn as nn
import cv2
import numpy as np
import json
import matplotlib.pyplot as plt
import pandas as pd
import math
from scipy.spatial.transform import Rotation as R

In [2]:
from calc_pixel import *
import gps_utils as gutils

In [3]:
with open('camera_info.json') as f:
    camera_info = json.load(f)

In [4]:
def gps2cart(point):
    x, y = gutils.to_pixel(point[0], point[1], center_x, center_y, zoom=18, width=1024, height=1024)
    output = torch.tensor([x,y,point[-1]])
    return output

In [5]:
center_x, center_y = 39.351, -76.345

In [6]:
img_h, img_w = 1080, 1920
fov_h, fov_v = 84, 53  # degrees

n_pos, n_cov = parse_log('Node Positioning GPS Data/R00-node1-left.log')
n_pos = gps2cart(n_pos)
rot_df = pd.read_csv('20240108-224709_camera_rotation.csv', header=None)
rot = R.from_euler('xyz', rot_df.mean())
n_rot = torch.from_numpy(rot.as_matrix()).float()
n_pos = n_pos.unsqueeze(0)
#n_rot = n_rot.unsqueeze(0)

fnames = ['samples/frame_1800.jpg', 'samples/frame_2800.jpg', 'samples/frame_4440.jpg']
frames = [load_frame(fname) for fname in fnames]
#pixels = np.array([detect(frame) for frame in frames])

In [14]:
print(n_pos)

tensor([[539.0000, 605.0000,   9.1421]])


In [7]:
obj_points = []
for i in [1,2,3]:
    o_pos, o_cov = parse_log('Node Positioning GPS Data/R00-node1-pos%d.log' % i)
    o_pos = gps2cart(o_pos)
    obj_points.append(o_pos)
obj_points = torch.stack(obj_points)

In [8]:
pitch, yaw, roll = rot_df.mean()
pitch = torch.tensor(pitch).unsqueeze(0)
yaw = torch.tensor(yaw).unsqueeze(0)
roll = torch.tensor(roll).unsqueeze(0)

In [9]:
class Euler2RotationMatrix(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, roll, pitch, yaw):
        # angles is expected to be a batch of N x 3 (roll, pitch, yaw)
        #roll, pitch, yaw = angles[:, 0], angles[:, 1], angles[:, 2]

        # Precompute cosines and sines of the angles
        cos_r, sin_r = torch.cos(roll), torch.sin(roll)
        cos_p, sin_p = torch.cos(pitch), torch.sin(pitch)
        cos_y, sin_y = torch.cos(yaw), torch.sin(yaw)

        # Construct rotation matrix for roll
        r_mat = torch.stack([
            torch.ones_like(roll), torch.zeros_like(roll), torch.zeros_like(roll),
            torch.zeros_like(roll), cos_r, -sin_r,
            torch.zeros_like(roll), sin_r, cos_r
        ], dim=-1).view(-1, 3, 3)

        # Construct rotation matrix for pitch
        p_mat = torch.stack([
            cos_p, torch.zeros_like(pitch), sin_p,
            torch.zeros_like(pitch), torch.ones_like(pitch), torch.zeros_like(pitch),
            -sin_p, torch.zeros_like(pitch), cos_p
        ], dim=-1).view(-1, 3, 3)

        # Construct rotation matrix for yaw
        y_mat = torch.stack([
            cos_y, -sin_y, torch.zeros_like(yaw),
            sin_y, cos_y, torch.zeros_like(yaw),
            torch.zeros_like(yaw), torch.zeros_like(yaw), torch.ones_like(yaw)
        ], dim=-1).view(-1, 3, 3)

        # Combine the rotation matrices
        rotation_matrix = torch.bmm(torch.bmm(y_mat, p_mat), r_mat)

        return rotation_matrix

In [10]:
def in_bounds(pixel, image_width=1920, image_height=1080):
    x, y = pixel
    return 0 <= x < image_width and 0 <= y < image_height

In [11]:
new_frames = []
euler2rot = Euler2RotationMatrix()
yaw_values = torch.arange(0, 2*math.pi, 0.1)
for i, frame in enumerate(frames):
    new_frame = frame.copy()
    obj_point = obj_points[i].unsqueeze(0)
    for yaw in yaw_values:
        yaw = yaw.unsqueeze(0)
        rot_matrix = euler2rot(roll,pitch,yaw)[0]
        local_points = (obj_point - n_pos) @ rot_matrix
        X = (local_points[:, 1] + 0)
        Y = (local_points[:, 2] + 0)
        Z = (local_points[:, 0]) + 0
        u = (X/Z) * camera_info['left']['fx'] + camera_info['left']['cx']
        v = (Y/Z) * camera_info['left']['fy'] + camera_info['left']['cy']
        #pixel = torch.stack((u, v), dim=1).to(int)
        #print(pixel.shape)
        pixel = (int(u.item()), int(v.item()))
        if in_bounds(pixel):
            new_frame = cv2.circle(new_frame, tuple(pixel), 10, (0, 255, 0), -1)
            new_frame = cv2.putText(new_frame, "{:.2f}".format(yaw.item()), (pixel[0], pixel[1] - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    new_frames.append(new_frame)

In [12]:
for i, frame in enumerate(new_frames):
    cv2.imwrite('results/frame_%d.png' % i, frame)